In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import f1_score, classification_report, confusion_matrix
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from xgboost import XGBClassifier,XGBRFClassifier, XGBRegressor, XGBRFRegressor
from sklearn.feature_selection import RFE
from sklearn.svm import SVC
from importlib import reload
from pactools.grid_search import GridSearchCVProgressBar
import sys

training_data = pd.read_csv('2024-introduction-to-data-analysis-hw-5-2/train.csv')
testing_data = pd.read_csv('2024-introduction-to-data-analysis-hw-5-2/test.csv')
training_data.drop(columns=["SEASON_ID", "TEAM_ID", 'position'], inplace=True)
testing_data.drop(columns=["ID", "SEASON_ID", "TEAM_ID"], inplace=True)

training_data.dropna(subset=["MIN"], inplace=True)
training_data.fillna(0, inplace=True)

X = training_data.drop(columns=["MIN"])
y = training_data["MIN"]

params = {
    "n_estimators": [10, 50, 100, 200, 500],
    "max_depth": [None, 10, 20, 50, 100],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "max_features": [1, "sqrt", "log2"]
}

model = RandomForestRegressor()
grid = RandomizedSearchCV(model, params, n_iter=100, cv=5, verbose=1, n_jobs=-1, scoring="neg_mean_squared_error")
grid.fit(X, y)
print(grid.best_params_)
model = grid.best_estimator_
# model.fit(X, y)

y_pred = model.predict(testing_data)
submission_df = pd.DataFrame()
temp = pd.read_csv('2024-introduction-to-data-analysis-hw-5-2/test.csv')
submission_df["ID"] = temp["ID"]
submission_df["MIN"] = y_pred
print(submission_df)
submission_df.to_csv("test_submission.csv", index=False)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
{'n_estimators': 500, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'log2', 'max_depth': None}
        ID       MIN
0        1    40.554
1        2  2368.948
2        3  1652.484
3        4   454.422
4        5   128.192
...    ...       ...
1995  1996  2598.434
1996  1997  1103.126
1997  1998   978.408
1998  1999  1095.178
1999  2000   375.092

[2000 rows x 2 columns]
